In [1]:
import json
import pandas as pd

In [2]:
# Load JSON
with open('data/raw/abstracts_sessions_authors_topics.json') as f:
    data = json.load(f)

# Convert to DataFrame
df_raw = pd.DataFrame(data)

In [3]:
# Quick overview
print(df_raw.info())
print(df_raw.describe(include='all'))  # includes categorical stats
print(df_raw.head())

# Example: count by category
if 'category' in df_raw.columns:
    print(df_raw['category'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7604 entries, 0 to 7603
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                7604 non-null   int64 
 1   title             7604 non-null   object
 2   submission_date   7604 non-null   object
 3   publication_date  0 non-null      object
 4   language_ref      7604 non-null   int64 
 5   content_raw       7598 non-null   object
 6   content           7604 non-null   object
 7   word_count        0 non-null      object
 8   keywords          0 non-null      object
 9   topic_id          7604 non-null   int64 
 10  topic_title       7604 non-null   object
 11  session_id        7604 non-null   int64 
 12  session_title     7604 non-null   object
 13  authors           7604 non-null   object
dtypes: int64(4), object(10)
memory usage: 831.8+ KB
None
                 id                               title      submission_date  \
count   7604.000000

In [4]:
print(df_raw['authors'].iloc[0])

[{'inserted': '2017-08-30T18:21:49', 'updated': '2025-04-06T20:19:55', 'owner_ref': 0, 'last_editor_ref': 2, 'state_key': '', 'deleted': 0, 'version_number': 1, 'transaction_number': 0, 'sequence': 1, 'participant_ref': 2869, 'academicdegree': 'Prof. Dr. Dr.', 'affiliationorganisation': '2869', 'affiliationcity': 'Aachen', 'affiliationcountry_ref': 83, 'external_identifiers': None, 'cpo__co2_id': None, 'cpo__sinner_id': 511, 'author_id': 10}]


In [5]:
# CHECK AUTHOR DUPLICATES
# Step 1: Make a shallow copy of the authors column only (won't affect the original df)
authors_col = df_raw["authors"]

# Step 2: Check length of each list (i.e., number of authors)
author_counts = authors_col.apply(lambda x: len(x) if isinstance(x, list) else 0)

# Step 3: Analyze
has_multiple_authors = (author_counts > 1).any()
num_entries_with_multiple_authors = (author_counts > 1).sum()

print("Any entries with multiple authors:", has_multiple_authors)
print("Number of such entries:", num_entries_with_multiple_authors)


Any entries with multiple authors: False
Number of such entries: 0


In [6]:
def convert_object_columns_to_string(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            # Drop nulls and check if all remaining values are strings
            non_null = df[col].dropna()
            if non_null.map(type).eq(str).all():
                df[col] = df[col].astype('string')
    return df

# Filter out rows with empty author lists before exploding
df_temp = df_raw[df_raw['authors'].map(lambda x: len(x) > 0 if isinstance(x, list) else False)].copy()

# Explode the lists so each author dict becomes its own row
df_exploded = df_temp.explode('authors').reset_index(drop=True)

# Normalize the dicts into columns
authors_normalized = pd.json_normalize(df_exploded['authors'])

# Combine back with any submission-level info if needed:
# e.g., keep submission id or title alongside author details
df = pd.concat(
    [df_exploded.drop(columns=['authors']).reset_index(drop=True),
     authors_normalized.reset_index(drop=True)],
    axis=1
)

print(df["external_identifiers"].value_counts())

columns_to_drop = ['submission_date', 'publication_date', 'content', 'word_count',
                   'keywords', 'inserted', 'updated', 'owner_ref', 'last_editor_ref',
                   'state_key', 'deleted', 'version_number', 'transaction_number',
                   'sequence', 'external_identifiers', 'cpo__co2_id', 'cpo__sinner_id']
df = df.drop(columns = columns_to_drop, axis=1)

df = convert_object_columns_to_string(df)

Series([], Name: count, dtype: int64)


In [7]:
# Convert "language_ref" to "language" (int to String)
df.rename(columns={'language_ref': 'language'}, inplace=True)

language_map = {
    254: 'German',
    255: 'English'
}

df['language'] = df['language'].map(language_map)
df['language'] = df['language'].astype('string')

In [8]:
# Check country codes

# Drop rows where either city or country_ref is missing
df_geo = df.dropna(subset=['affiliationcity', 'affiliationcountry_ref'])

# Group by country code and city, count occurrences
city_counts = (
    df_geo.groupby(['affiliationcountry_ref', 'affiliationcity'])
    .size()
    .reset_index(name='count')
)

# For each country code, get the top 2 most common cities
top_cities_by_country = (
    city_counts
    .sort_values(['affiliationcountry_ref', 'count'], ascending=[True, False])
    .groupby('affiliationcountry_ref')
    .head(2)
)

# Print result nicely
for country_code, group in top_cities_by_country.groupby('affiliationcountry_ref'):
    print(f"Country code {int(country_code)}:")
    for _, row in group.iterrows():
        print(f"  {row['affiliationcity']} ({row['count']} times)")
    print()


Country code 3:
  Tirana (8 times)
  Lezhe (1 times)

Country code 4:
  Annaba (1 times)

Country code 11:
  Buenos Aires (51 times)
  Rosario (5 times)

Country code 12:
  Yerevan (3 times)

Country code 14:
  Sydney (40 times)
  Melbourne (20 times)

Country code 15:
  Vienna (86 times)
  Wien (38 times)

Country code 16:
  Bakou (3 times)

Country code 18:
  Manama (3 times)

Country code 19:
  Dhaka (5 times)
  Rajshahi (1 times)

Country code 21:
  Minsk (8 times)

Country code 22:
  Kortenberg (9 times)
  Brussels (7 times)

Country code 29:
  Tuzla (7 times)
  Sarajevo (5 times)

Country code 32:
  São Paulo (29 times)
  Porto Alegre (23 times)

Country code 35:
  Plovdiv (4 times)

Country code 41:
  Toronto (56 times)
  Vancouver (32 times)

Country code 45:
  Santiago de Chile (23 times)
  Santiago (7 times)

Country code 46:
  Beijing (9 times)
  Shanghai (8 times)

Country code 49:
  Medellin (9 times)
  Bogota (5 times)

Country code 54:
  San Jose (5 times)

Country code 

In [9]:
country_code_map = {3: "Albania", 4: "Algeria", 11: "Argentina", 12: "Armenia", 14: "Australia",
                    15: "Austria", 16: "Azerbaijan", 18: "Bahrain", 19: "Bangladesh", 21: "Belarus",
                    22: "Belgium", 29: "Bosnia and Herzegovina", 32: "Brazil", 35: "Bulgaria",
                    41: "Canada", 45: "Chile", 46: "China", 49: "Colombia", 54: "Costa Rica",
                    56: "Croatia", 57: "Cuba", 59: "Cyprus", 60: "Czech Republic", 61: "Denmark",
                    64: "Dominican Republic", 65: "Ecuador", 66: "Egypt", 67: "El Salvador", 70: "Estonia",
                    75: "Finland", 76: "France", 82: "Georgia", 83: "Germany", 84: "Ghana", 86: "Greece",
                    87: "Unknown", 91: "Unknown", 100: "Hong Kong", 101: "Hungary", 102: "Iceland",
                    103: "India", 104: "Indonesia", 105: "Iran", 107: "Ireland", 109: "Israel",
                    110: "Italy", 112: "Japan", 115: "Unknown", 116: "Kenya", 119: "South Korea",
                    121: "Kyrgyzstan", 123: "Latvia", 124: "Lebanon", 129: "Lithuania", 130: "Luxembourg",
                    131: "Macau", 132: "North Macedonia", 135: "Malaysia", 137: "Mali", 138: "Malta",
                    144: "Mexico", 146: "Moldova", 149: "Montenegro", 151: "Morocco", 152: "Mozambique",
                    153: "Myanmar", 156: "Nepal", 157: "Netherlands", 159: "New Zealand", 162: "Nigeria",
                    166: "Norway", 168: "Pakistan", 170: "Palestine", 171: "Panama", 173: "Paraguay",
                    174: "Peru", 175: "Philippines", 177: "Poland", 178: "Portugal", 180: "Qatar",
                    182: "Romania", 183: "Russia", 197: "Serbia", 200: "Singapore", 202: "Unknown",
                    203: "Slovenia", 206: "South Africa", 209: "Spain", 210: "Sri Lanka", 211: "Unknown",
                    215: "Sweden", 216: "Switzerland", 218: "Taiwan", 221: "Thailand", 227: "Tunisia",
                    228: "Turkey", 232: "Uganda", 233: "Ukraine", 234: "United Arab Emirates",
                    235: "United Kingdom", 236: "United States", 239: "Uzbekistan", 241: "Venezuela"
}

df = df.rename(columns={'affiliationcountry_ref': 'affiliationcountry'})

df['affiliationcountry'] = (
    df['affiliationcountry'].map(country_code_map).astype('string')
)

In [10]:
print(df.info())
print(df.describe(include='all'))  # includes categorical stats
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7581 entries, 0 to 7580
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       7581 non-null   int64 
 1   title                    7581 non-null   string
 2   language                 7581 non-null   string
 3   content_raw              7575 non-null   string
 4   topic_id                 7581 non-null   int64 
 5   topic_title              7581 non-null   string
 6   session_id               7581 non-null   int64 
 7   session_title            7581 non-null   string
 8   participant_ref          7581 non-null   int64 
 9   academicdegree           7581 non-null   string
 10  affiliationorganisation  7581 non-null   string
 11  affiliationcity          7581 non-null   string
 12  affiliationcountry       7575 non-null   string
 13  author_id                7581 non-null   int64 
dtypes: int64(5), string(9)
memory usage: 829

In [11]:
print(df['participant_ref'].value_counts(dropna=False))

participant_ref
2966    22
2636    21
5799    21
1413    20
3073    20
        ..
7573     1
7570     1
7521     1
7518     1
7751     1
Name: count, Length: 3112, dtype: int64


In [12]:
## TROUBLESHOOTING
# Check raw authors field in the original JSON-parsed data for a row with NaN
# df_raw.loc[nan_participants.index[0], 'authors']
# Was 'participant_ref' present in all authors before flattening?
df_raw['authors'].apply(lambda lst: all('participant_ref' in d for d in lst if isinstance(d, dict))).value_counts()

authors
True    7604
Name: count, dtype: int64

In [13]:
# CHECK ABSTRACTS IN MULTIPLE SESSIONS. ONLY "CLOSING STATEMENT"
# Step 1: Group by abstract ID and count unique session IDs
session_counts = df.groupby('id')['session_id'].nunique()

# Step 2: Filter for abstracts linked to more than one session
multi_session_ids = session_counts[session_counts > 1].index

# Step 3: Display those abstracts with their session IDs
conflicts = df[df['id'].isin(multi_session_ids)][['id', 'session_id']].drop_duplicates()

# Print result
if not conflicts.empty:
    print("Abstracts linked to multiple sessions:")
    print(conflicts.sort_values(by='id'))
else:
    print("No abstracts are linked to multiple sessions.")

Abstracts linked to multiple sessions:
        id  session_id
5445  4866         570
6608  4866         859


In [14]:
# HANDLE DUPLICATES
# Count total number of rows
total_rows = len(df)
print(f"Total rows: {total_rows}")

# Count true duplicate rows (i.e., every value across all columns is identical)
duplicate_mask = df.duplicated(keep='first')
num_duplicates = duplicate_mask.sum()
print(f"Number of true duplicate rows: {num_duplicates}")

# drop
df = df.drop_duplicates()
df = df.reset_index(drop=True)

Total rows: 7581
Number of true duplicate rows: 57


In [15]:
# Fix missing values

# All missing in "affiliationcountry" are Kosovo
print(df[df["affiliationcountry"].isna()]["affiliationcity"])
df["affiliationcountry"] = df["affiliationcountry"].fillna("Kosovo")

# Replace null with "" in "content_raw"
df["content_raw"] = df["affiliationcountry"].fillna("")

3953    Prishtina
3954    Prishtina
3955    Prishtina
6001      Prizren
7057    Prishtina
7386      Prizren
Name: affiliationcity, dtype: string


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7524 entries, 0 to 7523
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       7524 non-null   int64 
 1   title                    7524 non-null   string
 2   language                 7524 non-null   string
 3   content_raw              7524 non-null   string
 4   topic_id                 7524 non-null   int64 
 5   topic_title              7524 non-null   string
 6   session_id               7524 non-null   int64 
 7   session_title            7524 non-null   string
 8   participant_ref          7524 non-null   int64 
 9   academicdegree           7524 non-null   string
 10  affiliationorganisation  7524 non-null   string
 11  affiliationcity          7524 non-null   string
 12  affiliationcountry       7524 non-null   string
 13  author_id                7524 non-null   int64 
dtypes: int64(5), string(9)
memory usage: 823